1. Implemente un perceptrón simple que aprenda la función lógica $AND$ y la función lógica $OR$, de $2$ y de $4$ entradas. Muestre la evolución del error durante el entrenamiento. Para el caso de $2$ dimensiones, grafique la recta discriminadora y todos los vectores de entrada de la red

![](img/perceptrón-simple1.png)

$AND$ de $2$ entradas:
| $x_1$ | $0$ | $0$ | $1$ | $1$ |
|-------|-----|-----|-----|-----|
| $x_2$ | $0$ | $1$ | $0$ | $1$ |
| $y$   | $0$ | $0$ | $0$ | $1$ |

In [1]:
import numpy as np
from matplotlib import pyplot as plt
np.random.seed(2002)

In [2]:
def AND(X):
    return all(X)

def OR(X):
    return any(X)

In [3]:
class PerceptronSimple:
    def __init__(self):
        self.W = np.random.randn(3)
    def train(self, X, Y, alpha, iter_):
        for _ in range(iter_):
            for n in range(len(X)):
                a = self.predict(X[n])
                if a != Y[n]:
                    self.W[0] += alpha * (Y[n] - a) * X[n][0]
                    self.W[1] += alpha * (Y[n] - a) * X[n][1]
                    self.W[2] += alpha * (Y[n] - a) * (-1)
    def predict(self, x):
        h = np.dot(np.append(x, -1), self.W)
        return 0 if h < 0 else 1

In [4]:
X_train = [[x1,x2] for x1 in [0,1] for x2 in[0,1]]
Y_train = [AND(x) for x in X_train]

perceptron = PerceptronSimple()
perceptron.train(X_train[1:] + [[1,1]], Y_train[1:] + [True], 0.01, 10000)

for x in X_train:
    print(x, perceptron.predict(x))

[0, 0] 0
[0, 1] 0
[1, 0] 0
[1, 1] 1


$AND$ de $4$ entradas:

| $x_1$ | $0$ | $0$ | $0$ | $0$ | $0$ | $0$ | $0$ | $0$ | $1$ | $1$ | $1$ | $1$ | $1$ | $1$ | $1$ | $1$ |
|-------|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|
| $x_2$ | $0$ | $0$ | $0$ | $0$ | $1$ | $1$ | $1$ | $1$ | $0$ | $0$ | $0$ | $0$ | $1$ | $1$ | $1$ | $1$ |
| $x_3$ | $0$ | $0$ | $1$ | $1$ | $0$ | $0$ | $0$ | $1$ | $0$ | $0$ | $1$ | $1$ | $0$ | $0$ | $1$ | $1$ |
| $x_4$ | $0$ | $1$ | $0$ | $1$ | $0$ | $1$ | $0$ | $1$ | $0$ | $1$ | $0$ | $1$ | $0$ | $1$ | $0$ | $1$ |
| $y$   | $0$ | $0$ | $0$ | $0$ | $0$ | $0$ | $0$ | $0$ | $0$ | $0$ | $0$ | $0$ | $0$ | $0$ | $0$ | $1$ |

In [5]:
class PerceptronSimple:
    def __init__(self):
        self.W = np.random.randn(5)
    def train(self, X, Y, alpha, iter_):
        for _ in range(iter_):
            for n in range(len(X)):
                a = self.predict(X[n])
                if a != Y[n]:
                    self.W[0] += alpha * (Y[n] - a) * X[n][0]
                    self.W[1] += alpha * (Y[n] - a) * X[n][1]
                    self.W[2] += alpha * (Y[n] - a) * X[n][2]
                    self.W[3] += alpha * (Y[n] - a) * X[n][3]
                    self.W[4] += alpha * (Y[n] - a) * (-1)
    def predict(self, x):
        h = np.dot(np.append(x, -1), self.W)
        return 0 if h < 0 else 1

In [6]:
X_train = [[x1,x2,x3,x4] for x1 in [0,1] for x2 in[0,1] for x3 in [0,1] for x4 in[0,1]]
Y_train = [AND(x) for x in X_train]

perceptron = PerceptronSimple()
perceptron.train(X_train[5:] + [[1,1,1,1] for _ in range(5)], Y_train[5:] + [True for _ in range(5)], 0.001, 10000)

for x in X_train:
    print(x, perceptron.predict(x))

[0, 0, 0, 0] 0
[0, 0, 0, 1] 0
[0, 0, 1, 0] 0
[0, 0, 1, 1] 0
[0, 1, 0, 0] 0
[0, 1, 0, 1] 0
[0, 1, 1, 0] 0
[0, 1, 1, 1] 0
[1, 0, 0, 0] 0
[1, 0, 0, 1] 0
[1, 0, 1, 0] 0
[1, 0, 1, 1] 0
[1, 1, 0, 0] 0
[1, 1, 0, 1] 0
[1, 1, 1, 0] 0
[1, 1, 1, 1] 1


2. Implemente un perceptrón multicapa que aprenda la función lógica $XOR$ de $2$ y de $4$ entradas (utilizando el algoritmo Backpropagation y actualizando en batch). Muestre cómo evoluciona el error durante el entrenamiento.

![](img/perceptrón-multicapa1.png)

$XOR$ de $2$ entradas:
| $x_1$ | $0$ | $0$ | $1$ | $1$ |
|-------|-----|-----|-----|-----|
| $x_2$ | $0$ | $1$ | $0$ | $1$ |
| $y$   | $0$ | $1$ | $1$ | $0$ |

In [241]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def d_sigmoid(z):
    return sigmoid(z) * (1 - sigmoid(z))

def tanh(z):
    return np.tanh(z)

def d_tanh(z):
    return 1 - np.tanh(z)**2

class PerceptronMulticapa:
    def __init__(self, sizes, g=sigmoid, d_g=d_sigmoid):
        self.L = len(sizes)
        self.sizes = sizes
        self.a = [[0 for _ in range(s)] for s in sizes]
        self.z = [[0 for _ in range(s)] for s in sizes[1:]]
        self.w = [np.random.randn(n,m) for n, m in zip(sizes[1:], sizes[:-1])]
        self.b = [np.random.randn(s) for s in sizes[1:]]
        self.g = g
        self.d_g = d_g
    def predict(self, x):
        self.a[0] = x
        for l in range(1, self.L):
            self.z[l-1] = self.w[l-1] @ self.a[l-1] + self.b[l-1]
            self.a[l] = self.g(self.z[l-1])
        return self.a[-1]

    def train(self, X, Y, lr=0.01, iters=1000):
        for _ in range(iters):  # Iterar varias veces
            for x, y in zip(X, Y):
                # predigo x para actualizar la matriz de activaciones
                self.predict(x)
                
                # calculo el error
                grad_C_a = self.a[-1] - y
                delta_l = grad_C_a * self.d_g(self.z[-1])
                
                # inicializo los gradientes
                grad_C_w = [np.zeros_like(w) for w in self.w]
                grad_C_b = [np.zeros_like(b) for b in self.b]
                
                # backprop
                for l in range(self.L-2, -1, -1):
                    grad_C_w[l] = np.outer(delta_l, self.a[l])
                    grad_C_b[l] = delta_l
                    if l > 0:
                        delta_l = (self.w[l].T @ delta_l) * self.d_g(self.z[l-1])
                
                # muevo los parámetros en la dirección contraria al gradiente en módulo learning rate
                self.w = [w - lr * grad_w for w, grad_w in zip(self.w, grad_C_w)]
                self.b = [b - lr * grad_b for b, grad_b in zip(self.b, grad_C_b)]

In [208]:
X_train = [[x1, x2] for x1 in [0, 1] for x2 in [0, 1]]
Y_train = [x1 ^ x2 for x1, x2 in X_train]

perceptron = PerceptronMulticapa([2,3,1])
perceptron.train(X_train, Y_train, lr=0.1, iters=10000)
for x in X_train:
    print(x, 1 if perceptron.predict(x) > .5 else 0)

[0, 0] 0
[0, 1] 1
[1, 0] 1
[1, 1] 0


$XOR$ de $4$ entradas:
| $x_1$ | $0$ | $0$ | $0$ | $0$ | $0$ | $0$ | $0$ | $0$ | $1$ | $1$ | $1$ | $1$ | $1$ | $1$ | $1$ | $1$ |
|-------|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|-----|
| $x_2$ | $0$ | $0$ | $0$ | $0$ | $1$ | $1$ | $1$ | $1$ | $0$ | $0$ | $0$ | $0$ | $1$ | $1$ | $1$ | $1$ |
| $x_3$ | $0$ | $0$ | $1$ | $1$ | $0$ | $0$ | $1$ | $1$ | $0$ | $0$ | $1$ | $1$ | $0$ | $0$ | $1$ | $1$ |
| $x_4$ | $0$ | $1$ | $0$ | $1$ | $0$ | $1$ | $0$ | $1$ | $0$ | $1$ | $0$ | $1$ | $0$ | $1$ | $0$ | $1$ |
| $y$   | $0$ | $1$ | $1$ | $0$ | $1$ | $0$ | $0$ | $1$ | $1$ | $0$ | $0$ | $1$ | $0$ | $1$ | $1$ | $0$ |

In [182]:
def xor(x):
    return x[0] ^ x[1] ^ x[2] ^ x[3]

X_train = [[x1, x2, x3, x4] for x1 in [0, 1] for x2 in [0, 1] for x3 in [0, 1] for x4 in [0, 1]]
Y_train = [xor(x) for x in X_train]

perceptron = PerceptronMulticapa([4,8,1])
perceptron.train(X_train,Y_train,lr=0.01,iters=100000)
for x in X_train:
    print(x, 1 if perceptron.predict(x) > .5 else 0, xor(x))

[0, 0, 0, 0] 0 0
[0, 0, 0, 1] 1 1
[0, 0, 1, 0] 1 1
[0, 0, 1, 1] 0 0
[0, 1, 0, 0] 1 1
[0, 1, 0, 1] 0 0
[0, 1, 1, 0] 0 0
[0, 1, 1, 1] 1 1
[1, 0, 0, 0] 1 1
[1, 0, 0, 1] 0 0
[1, 0, 1, 0] 0 0
[1, 0, 1, 1] 1 1
[1, 1, 0, 0] 0 0
[1, 1, 0, 1] 1 1
[1, 1, 1, 0] 1 1
[1, 1, 1, 1] 0 0


4.
    a) Implemente una red con aprendizaje Backpropagation que aprenda la siguiente función:
    $$
    f(x, y, z) = \sin(x) + \cos(y) + z
    $$
    donde $x, y \in [0, 2\pi]$ y $z \in [-1, 1]$.  
    Para ello construya un conjunto de datos de entrenamiento y un conjunto de evaluación. Muestre la evolución del error de entrenamiento y de evaluación en función de las épocas de entrenamiento.

    b) Estudie la evolución de los errores durante el entrenamiento de una red con una capa oculta de $30$ neuronas cuando el conjunto de entrenamiento contiene $40$ muestras.  
    ¿Qué ocurre si el minibatch tiene tamaño 40? ¿Y si tiene tamaño 1?

In [237]:
def f(x, y, z):
    return np.sin(x) + np.cos(y) + z

X_train = [[x, y, z] for x, y, z in zip(np.linspace(0, 2*np.pi, 100), np.linspace(0, 2*np.pi, 100), np.linspace(-1, 1, 100))]
Y_train = np.array([f(x, y, z) for x, y, z in X_train])

In [242]:
# voy a tener que cambiar la función costo usando el mse
# la arquitectura no debe estar tan mal
# me está errando los negativos porque la sigmoide sólo me da > 0
# voy a usar tangente hiperbólica
# la tanh por lo menos no es > 0 pero está acotada < 1 :(

perceptron = PerceptronMulticapa([3,15,15,15,1], g=tanh, d_g=d_tanh)
perceptron.train(X_train,Y_train,lr=0.01,iters=10000)

# for x, y, z in X_train:
#     print(str(round(x, 2))+"\t"+
#           str(round(y, 2))+"\t"+
#           str(round(z, 2))+"\t"+
#           str(round(perceptron.predict([x, y, z])[0],2))+"\t"+
#           str(round(f(x, y, z),2)))

In [243]:
def mse(Y_h, Y):
    return sum((Y_h - Y)**2)/len(Y)

Y = np.array([perceptron.predict([x, y, z])[0] for x, y, z in X_train])
mse(Y_train, Y)

0.04492634565350915

In [244]:
for i in range(len(Y)):
    print(round(Y[i],2), round(Y_train[i],2))

-0.0 0.0
0.09 0.08
0.16 0.16
0.23 0.23
0.3 0.3
0.37 0.36
0.42 0.42
0.48 0.47
0.52 0.52
0.56 0.56
0.6 0.6
0.63 0.63
0.66 0.66
0.68 0.68
0.69 0.69
0.7 0.7
0.7 0.7
0.69 0.7
0.68 0.69
0.67 0.67
0.65 0.66
0.63 0.63
0.61 0.6
0.57 0.57
0.54 0.53
0.49 0.49
0.45 0.44
0.39 0.39
0.34 0.34
0.28 0.28
0.21 0.22
0.15 0.16
0.09 0.1
0.03 0.03
-0.03 -0.03
-0.1 -0.1
-0.16 -0.17
-0.23 -0.24
-0.3 -0.31
-0.38 -0.38
-0.45 -0.45
-0.52 -0.52
-0.59 -0.58
-0.65 -0.65
-0.7 -0.71
-0.76 -0.77
-0.82 -0.83
-0.89 -0.88
-0.94 -0.93
-0.97 -0.98
-0.99 -1.02
-1.0 -1.06
-1.0 -1.09
-1.0 -1.13
-1.0 -1.15
-1.0 -1.17
-1.0 -1.19
-1.0 -1.2
-1.0 -1.2
-1.0 -1.2
-1.0 -1.19
-1.0 -1.18
-1.0 -1.16
-1.0 -1.14
-1.0 -1.11
-1.0 -1.07
-0.99 -1.03
-0.97 -0.99
-0.94 -0.93
-0.9 -0.88
-0.84 -0.82
-0.77 -0.75
-0.69 -0.68
-0.6 -0.6
-0.52 -0.52
-0.43 -0.44
-0.34 -0.35
-0.25 -0.26
-0.15 -0.16
-0.05 -0.06
0.04 0.04
0.14 0.14
0.23 0.25
0.32 0.35
0.43 0.46
0.56 0.57
0.71 0.68
0.85 0.79
0.95 0.9
0.99 1.01
1.0 1.12
1.0 1.23
1.0 1.33
1.0 1.44
1.0 1.54
1

5. Siguiendo el trabajo de Hinton y Salakhutdinov (2006), entrene una máquina restringida
de Boltzmann con imágenes de la base de datos MNIST. Muestre el error de
recontruccion durante el entrenamiento, y ejemplos de cada uno de los dígitos
reconstruidos.

In [ ]:
from itertools import product
from scipy.io import loadmat

train = loadmat("data/rbm/datosTrain.mat")["data"]
test = loadmat("data/rbm/datosTest.mat")["data"]

v_size = 784
h_size = 100
L = 2

# inicializo la red
v = np.array([0 for _ in range(v_size)])
h = np.array([np.random.randn() for _ in range(h_size)])
w = np.random.randn(v_size, h_size)
bv = np.random.randn(v_size)
bh = np.random.randn(h_size)

def E(v, h):
    return -bv @ v -bh @ h - v @ w @ h

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# aumentando las probabilidades de los datos de entrenamiento
f_vh_data = np.zeros((v_size, h_size)) # fracción de veces que v_i y h_j están activados simultáneamente en los datos
f_vh_recon = np.zeros((v_size, h_size)) # misma fracción para las confabulaciones
for t in train:
    v = t
    p_h = sigmoid(bh + v @ w)
    h = np.array([1 if np.random.rand() < p else 0 for p in p_h])
    f_vh_data += np.outer(v, h)
    # confabulaciones
    p_v = sigmoid(bv + h @ w.T)
    v = np.array([1 if np.random.rand() < p else 0 for p in p_v])
    p_h = sigmoid(bh + v @ w)
    h = np.array([1 if np.random.rand() < p else 0 for p in p_h])
    f_vh_recon += np.outer(v, h)

f_vh_data /= len(train)
f_vh_recon /= len(train)

# acualizo los pesos
epsilon = 0.01
w += epsilon * (f_vh_data - f_vh_recon)

# tengo que actualizar los sesgos también

# unfolding
def mse(h, y):
    return sum((h - y)**2) / len(h)

/tmp/ipykernel_322644/707977394.py:18: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


6. Entrene una red convolucional para clasificar las imágenes de la base de datos MNIST.  
¿Cuál es la red convolucional más pequeña que puede conseguir con una exactitud de al menos 90% en el conjunto de evaluación? ¿Cuál es el perceptrón multicapa más
pequeño que puede conseguir con la misma exactitud?

7. Entrene un autoencoder para obtener una representación de baja dimensionalidad de las
imágenes de MNIST. Use dichas representaciones para entrenar un perceptrón
multicapa como clasificador. ¿Cuál es el tiempo de entrenamiento y la exactitud del
clasificador obtenido cuando parte de la representación del autoencoder, en
comparación con lo obtenido usando las imágenes originales?